# 01-02 : H2O 

## References

- [h2oai/h2o-danube3-500m-chat-GGUF](h2oai/h2o-danube3-500m-chat-GGUF)
- [foulds/h2o-danube3-500m-chat-llamafile](https://huggingface.co/foulds/h2o-danube3-500m-chat-llamafile/tree/main)

## 0. Download Llamafile

In [ ]:
!curl -L -o "../models/h2o-danube3-500m.llamafile" "https://huggingface.co/foulds/h2o-danube3-500m-chat-llamafile/resolve/main/h2o-danube3-500m-chat-Q5_K_M.llamafile?download=true"

In [ ]:
!chmod +x "../models/h2o-danube3-500m.llamafile"

## 1. Imports

In [1]:
import os
import subprocess
import stat
from IPython.display import display, Markdown, clear_output
from openai import OpenAI

## 2. Run Llamafile

In [2]:
model_file = "../models/h2o-danube3-500m.llamafile"

In [3]:
# Get the current working directory
cwd = os.getcwd()

# Construct the full path to the executable
executable_path = os.path.join(cwd, model_file)

# Ensure the file has execute permissions
if not os.access(executable_path, os.X_OK):
    st = os.stat(executable_path)
    os.chmod(executable_path, st.st_mode | stat.S_IEXEC)

# Define the arguments separately
arguments = ['--port', '8081', '--host', '0.0.0.0', '--nobrowser']

# Start the executable in the background
llamafile_process = subprocess.Popen(['bash', executable_path] + arguments)
print("Executable started in the background.")

Executable started in the background.


{"build":1500,"commit":"a30b324","function":"server_cli","level":"INFO","line":2869,"msg":"build info","tid":"34364426944","timestamp":1721307795}
{"function":"server_cli","level":"INFO","line":2872,"msg":"system info","n_threads":4,"n_threads_batch":-1,"system_info":"AVX = 0 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 0 | SSSE3 = 0 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | ","tid":"34364426944","timestamp":1721307795,"total_threads":8}


Apple Metal GPU support successfully loaded
llama_model_loader: loaded meta data with 28 key-value pairs and 147 tensors from h2o-danube3-500m-chat-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = danube3-500M
llama_model_loader: - kv   2:                          llama.block_count u32              = 16
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 1536
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 4096
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 16
llama_model_loader: - kv   7:  

{"function":"initialize","level":"INFO","line":489,"msg":"initializing slots","n_slots":1,"tid":"34364426944","timestamp":1721307796}
{"function":"initialize","level":"INFO","line":498,"msg":"new slot","n_ctx_slot":512,"slot_id":0,"tid":"34364426944","timestamp":1721307796}
{"function":"server_cli","level":"INFO","line":3090,"msg":"model loaded","tid":"34364426944","timestamp":1721307796}
{"function":"server_cli","hostname":"0.0.0.0","level":"INFO","line":3213,"msg":"HTTP server listening","port":"8081","tid":"34364426944","timestamp":1721307796}
{"function":"validate_model_chat_template","level":"ERR","line":478,"msg":"The chat template comes with this model is not yet supported, falling back to chatml. This may cause the model to output suboptimal responses","tid":"34364426944","timestamp":1721307796}
{"function":"update_slots","level":"INFO","line":1659,"msg":"all slots are idle and system prompt is empty, clear the KV cache","tid":"34364426944","timestamp":1721307796}



llama server listening at http://192.168.1.237:8081
llama server listening at http://127.0.0.1:8081



## 3. Test The Model

In [4]:
client = OpenAI(
    base_url="http://localhost:8081/v1",
    api_key = "sk-no-key-required"
)

In [5]:
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {"role": "user", "content": "Why is the sky blue"}
    ]
)
clear_output()
Markdown(completion.choices[0].message.content)

The sky is blue because it reflects sunlight. When sunlight hits the Earth's atmosphere, it scatters and refracts, causing the blue color of the sky. This is due to the scattering of shorter wavelengths of light, such as blue, by molecules and particles in the atmosphere. The scattering of longer wavelengths, like red and orange, is less pronounced, resulting in the blue color we see.</s>

### 3.1. Summarization Test

In [6]:
prompt = """
What is this text about?

TEXT:

Semantic search is a search engine technology that interprets the meaning 
of words and phrases. The results of a semantic search will return content
matching the meaning of a query, as opposed to content that literally 
matches words in the query.

Semantic search is a set of search engine capabilities, which includes 
understanding words from the searcher’s intent and their search context.

This type of search is intended to improve the quality of search results
by interpreting natural language more accurately and in context. Semantic
search achieves this by matching search intent to semantic meaning with
the help of technologies such as machine learning and artificial
intelligence.
"""

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    max_tokens=100,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

clear_output()
Markdown(completion.choices[0].message.content)


This text is about semantic search, a search engine technology that interprets the meaning of words and phrases. It is designed to improve the quality of search results by matching search intent to semantic meaning with the help of technologies such as machine learning and artificial intelligence.</s>

In [7]:
prompt = """
What is this customer complaint about?

Problems with the Vodabucks store with NO help and even managers that 
does not understand your problem. No resolution of my problems. The 
special on the vodabucks app does not correlate with the sms and voucher
received causing me to pay way more than what I would have paid without
using their voucher system
"""

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    max_tokens=256,
    temperature=0,
    top_p=0,
    messages=[
        {"role": "system", "content": "You are an customer support agent tasked with classifying complaints. You respond only with a single sentence after reviewing the text."},
        {"role": "user", "content": prompt}
    ]
)

clear_output()
Markdown(completion.choices[0].message.content)


Assistant: Of course! The customer complaint you're responding to is about the Vodabucks store with no customer support and managers who don't understand your problems. The customer is unhappy with the lack of assistance and the high prices they are paying for the vodabucks app. They also mention that the special on the vodabucks app does not correlate with the voucher they received, causing them to pay more than they would have without using their voucher system.</s>

## 10. Stop Llamafile

In [8]:
llamafile_process.terminate()